In [1]:
import pandas as pd
import re
import numpy as np
import functions

In [2]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x00000286D23A3598; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


# Tabla Detalle

In [9]:
def getProductName(row):
    if row['Velocity'] != '' and row['isNeba'] == 0 and row['Family'] != 'X FIBER':
        return row['Family'].title()+' '+row['Velocity']
    if row['Velocity'] != '' and row['isNeba'] == 0 and row['Family'] == 'X FIBER':
        return row['Family'].title()+' '+row['Velocity']+' - '+'own'
    if row['isNeba'] == 1:
        return row['Family'].title()+' '+row['Velocity']+' - '+'neba'
    if row['Family'] == 'X UCOM':
        return 'Teamwork+PBX'
    if row['Family'] == 'X MOBILE':
        return row['product_name'].title()
    else:
        return 'Assets'

def isNeba(description):
    if re.search('FIBER', description.upper()) and (re.search('NEBA', description.upper()) or re.search('IP', description.upper())) :
        return 1
    else:
        return 0

In [10]:
def Detalle(cur):
    cur.execute('''
            select segmento, product_name, companyname, sitename, created_as_trial, purchase_date,
            cancellation_date, service_date, rfb_date, deactivation_date, canal_venta, partnername,
            price, cif, assetid, asset_status, migrado, assetpadre_mig, assethijo_mig, rfb_migration,
            date_migration, purchase_migration
            from cartera_xbo
            where process_date = (
            select max(process_date) from cartera_xbo)
            and upper(product_name) not like '%CENTRALITA%'
            ''')
    df = pd.DataFrame(cur.fetchall())
    col_names = []
    for e in range(len(cur.description)):
        col_names.append(cur.description[e][0])
    df.rename(columns = dict(zip(list(range(33)), col_names)), inplace = True)
    df['Family'] = df.product_name.apply(lambda x: functions.getFamilyFrom(x))
    df['Velocity'] = df.product_name.apply(lambda x: functions.getVelocityFrom(x))
    df['isNeba'] = df.product_name.apply(lambda x: isNeba(x))
    df['product_name_ok'] = df.apply(lambda row: getProductName(row), axis = 1)
    df['cod_postal'] = ''
    df['precio_dto'] = ''
    df['cod_postal'] = ''
    df['%_dto'] = ''
    df['Segmento_Empleados'] = ''
    cols = ['segmento','Family', 'product_name', 'companyname', 'sitename', 'cod_postal',
        'created_as_trial', 'purchase_date', 'cancellation_date', 
        'service_date', 'rfb_date', 'deactivation_date', 'canal_venta', 
        'partnername', 'price','precio_dto', '%_dto', 'cif', 'Segmento_Empleados','assetid', 'asset_status', 'migrado', 
        'assetpadre_mig', 'assethijo_mig', 'rfb_migration', 'date_migration', 'purchase_migration','product_name_ok']
    df = df[cols]
    return df

In [11]:
prueba = Detalle(cur)

In [12]:
prueba.to_excel('./detalle_cartera_20200123_excel_3.xlsx',encoding='CP1252',index=False)

# Base

In [16]:
def canal(canal_venta):
    if re.search('DHO', canal_venta.upper()):
        return 'DHO'
    elif re.search('INTEGRA', canal_venta.upper()):
        return 'IT integrator'
    elif re.search('WEB', canal_venta.upper()):
        return 'WEB'

def rfb(row):
    return row['rfb_migration'] if row['migrado'] == '1' else row['rfb_date']
def purchase(row):
    return row['purchase_migration'] if row['migrado'] == '1' else row['purchase_date']
def purchaseToAsset(row):
    return 0 if row['rfb__c'] is pd.NaT else 1
def baja__c(row):
    return 1 if row['deactivation_date'] is not pd.NaT else 0
def cancelledSales(row):
    if row['cancellation_date'] is pd.NaT and row['rfb__c'] != pd.NaT:
        return 0
    else:
        return 1
def deactivation(row):
    return pd.NaT if row['isActive'] >= 1 else row['deactivation_date']

def Base(cur):
    cur.execute('''
            select segmento, assetid, cif, canal_venta, product_name, 
                    migrado,rfb_date, rfb_migration, deactivation_date, cancellation_date,
                    purchase_date, purchase_migration, deactivation_request_date
            from cartera_xbo
            where process_date = (
            select max(process_date) from cartera_xbo)
            and segmento in ('3-NOTRIAL')
            and upper(product_name) not like '%CENTRALITA%'
            ''')
    df = pd.DataFrame(cur.fetchall())
    col_names = []
    for e in range(len(cur.description)):
        col_names.append(cur.description[e][0])
    df.rename(columns = dict(zip(list(range(33)), col_names)), inplace = True)
    df['canal_venta'] = df.canal_venta.apply(lambda x: canal(x))
    df['rfb__c'] = df.apply(lambda row: rfb(row), axis = 1)
    df.drop(columns=['rfb_date','rfb_migration'], axis=1, inplace=True)
    df['purchase_date__c'] = df.apply(lambda row: purchase(row), axis = 1)
    df.drop(columns=['purchase_migration','purchase_date'], axis=1, inplace=True)
    df['Family'] = df.product_name.apply(lambda x: functions.getFamilyFrom(x))
    df['Velocity'] = df.product_name.apply(lambda x: functions.getVelocityFrom(x))
    df['isNeba'] = df.product_name.apply(lambda x: functions.isNeba(x))
    df['isSold'] = 1
    df['isAsset'] =  df.apply(lambda row: purchaseToAsset(row), axis = 1)
    df['isBaja'] = df.apply(lambda row: baja__c(row), axis = 1)
    df['isActive'] = df['isAsset'] - df['isBaja']
    df['isCancelled'] = df.apply(lambda row: cancelledSales(row), axis = 1)
    to_drop = df[(df['rfb__c'].isnull())&((df['deactivation_date'].notnull())|(df['deactivation_request_date'].notnull()))].index
    df.drop(to_drop , inplace=True)
    df.drop(columns=['deactivation_request_date'], axis=1, inplace=True)
    return df

def Cli_altas_bajas(df):
    df = df[df['rfb__c'].notnull()]  
    assets = df.isAsset.groupby(by=[df.cif,df.canal_venta]).sum().reset_index()
    assets.rename(columns={'isAsset':'assets'}, inplace=True)
    assets_act = df.isActive.groupby(by=[df.cif,df.canal_venta]).sum().reset_index()
    assets_act.rename(columns={'isActive':'assets_act'}, inplace=True)
    altas = df.rfb__c.groupby(by=[df.cif,df.canal_venta]).min().reset_index()
    bajas = df.deactivation_date.groupby(by=[df.cif,df.canal_venta]).max().reset_index()
    final = pd.merge(pd.merge(pd.merge(assets_act,altas,on=['cif','canal_venta']),bajas,on=['cif','canal_venta']),assets,on=['cif','canal_venta'])
    final['isClient'] = 1
    final['isActive'] = final.assets_act.apply(lambda x: 1 if x>=1 else 0)
    final['deactivation_date'] = final.apply(lambda row: deactivation(row), axis = 1)
    final['assets_dea'] = final['assets'] - final['assets_act']
    final['isBaja'] = -(final['isActive']-1)
    final.rename(columns={'assets':'assets_tot',
                      'rfb__c':'rfb',
                      }, inplace = True)
    return final

def Cli_altas_bajas_mes(altas_bajas_df):
    altas = altas_bajas_df[['isClient']].groupby(by=[altas_bajas_df.rfb]).sum()
    bajas = altas_bajas_df[['isBaja']].groupby(by=[altas_bajas_df.deactivation_date]).sum()
    bajas.index.name = 'rfb'
    altas_bajas = altas.join(bajas, how='outer').fillna(0.0)
    mes = altas_bajas.groupby(by=[altas_bajas.index.year, altas_bajas.index.month]).sum()
    mes.index.names = ['year', 'month']
    mes.reset_index(inplace=True)
    return mes

def Cli_altas_bajas_mes_canal(altas_bajas_df):
    altas = altas_bajas_df[['isClient']].groupby(by=[altas_bajas_df.rfb, altas_bajas_df.canal_venta]).sum()
    bajas = altas_bajas_df[['isBaja']].groupby(by=[altas_bajas_df.deactivation_date,altas_bajas_df.canal_venta]).sum()
    bajas.index.names = ['rfb','canal_venta']
    altas_bajas = altas.join(bajas, how='outer').fillna(0.0)
    mes_canal = altas_bajas.groupby(by=[altas_bajas.index.get_level_values('rfb').year, altas_bajas.index.get_level_values('rfb').month, altas_bajas.index.get_level_values('canal_venta')]).sum()
    mes_canal.index.names = ['year', 'month', 'canal']
    mes_canal.reset_index(inplace=True)
    return mes_canal

In [17]:
df = Base(cur)

In [12]:
print('Filtas Cartera:', len(df))
print('Total ventas:', len(df[df['isSold']==1]))
print('Total cancelaciones:', len(df[df['isCancelled']==1]))
print('Total assets:', len(df[df['isAsset']==1]))
print('Total bajas:', len(df[df['isBaja']==1]))
print('Total assets activos:', len(df[df['isActive']==1]))

Filtas Cartera: 1218
Total ventas: 1218
Total cancelaciones: 38
Total assets: 1111
Total bajas: 217
Total assets activos: 894


In [13]:
df.head(3)

,segmento,assetid,cif,canal_venta,product_name,migrado,deactivation_date,cancellation_date,rfb__c,purchase_date__c,Family,Velocity,isNeba,isSold,isAsset,isBaja,isActive,isCancelled
1,3-NOTRIAL,02i0N000006M6bnQAC,B85204915,DHO,X Music4B,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X MUSIC,,0,1,1,0,1,0
2,3-NOTRIAL,02i0N000006M6boQAC,B85204915,DHO,X Music4B,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X MUSIC,,0,1,1,0,1,0
3,3-NOTRIAL,02i0N000006M6bpQAC,B85204915,DHO,X Music4B,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X MUSIC,,0,1,1,0,1,0


# VENTAS

In [14]:
ass_vta_canal_fam_mes = df[['isSold']].groupby(by=[df.purchase_date__c.dt.year,df.purchase_date__c.dt.month,df.canal_venta,df.Family]).sum()
ass_vta_canal_fam_mes.index.names = ['purchase_year', 'purchase_month', 'canal_venta','Family']
ass_vta_canal_fam_mes.reset_index(inplace=True)
ass_vta_canal_fam_mes.pivot_table('isSold',['canal_venta','Family'],['purchase_year','purchase_month'],fill_value=0,margins=True,aggfunc=sum)

purchase_year              2018         2019                                \
purchase_month               10  11  12    1   2   3   4   5    6    7   8   
canal_venta   Family                                                         
DHO           X FIBER         0   3   3    4   8   1   2   9    5   13   3   
              X MUSIC         2   1   3    3   7   1   0   0    0    0   0   
              X PRIVACY       0   5   0    0  10  12   1  12   13    6   2   
              X PROTECTION    0   0   1    2   5  10   3   4   28   20   5   
              X SECURITY      0   0   0    0   0   0   0   2    2    2   0   
              X UCOM          0   5   0   19  21   1  12  18   56   50   7   
IT integrator X FIBER         0   1   4    3   0   1   6   4   11    3   1   
              X MUSIC         0   0   0    1   0   0   0   1    0    0   0   
              X PRIVACY       0   0   2    0   0   0   0   0    0    0   0   
              X PROTECTION    0   0   0    0   0   0   3   0    0    0   0   
              X SECURITY      0   0   2    2   0   0   0   3   11    3   0   
              X UCOM          0   0   6    0   1   1   3   1    0    0   0   
WEB           X FIBER         0   0   1    0   2   0   0   0    1    1   0   
              X MUSIC         1   1   0    0   0   1   0   5    0    0   0   
              X PRIVACY       0   0   2    0   0   0   2   0    0    0   0   
              X PROTECTION    0   0   3    0   0  15   7   0    1    0   0   
              X SECURITY      0   0   0    0   0   0   0   0    0    0   0   
              X UCOM          0   8   5    1   0   9   1   3    9    7   5   
All                           3  24  32   35  54  52  40  62  137  105  23   

purchase_year                                  2020   All  
purchase_month                9   10   11   12    1        
canal_venta   Family                                       
DHO           X FIBER         4    0    1    1    0    57  
              X MUSIC         3    0    0    0    1    21  
              X PRIVACY       9    6    0    2   11    89  
              X PROTECTION   13   27   22    4    1   145  
              X SECURITY     28    7    2    2    0    45  
              X UCOM         46   77   75  104   46   537  
IT integrator X FIBER        14   17    1    1    7    74  
              X MUSIC         1    0    0    0    0     3  
              X PRIVACY       0    0    0    0    0     2  
              X PROTECTION    6    0    4    0    0    13  
              X SECURITY     25   21    0    0    0    67  
              X UCOM          0    0    0    0    0    12  
WEB           X FIBER         2    0    0    1    0     8  
              X MUSIC         1    0    0    0    0     9  
              X PRIVACY       0    0    0    0    0     4  
              X PROTECTION    0    1    2    0    0    29  
              X SECURITY      0    0    0    0    1     1  
              X UCOM         13   17    3   16    5   102  
All                         165  173  110  131   72  1218

In [15]:
ass_vta_fam_mes = df[['isSold']].groupby(by=[df.purchase_date__c.dt.year,df.purchase_date__c.dt.month,df.Family]).sum()
ass_vta_fam_mes.index.names = ['purchase_year', 'purchase_month','Family']
ass_vta_fam_mes.reset_index(inplace=True)
ass_vta_fam_mes.pivot_table('isSold','Family',['purchase_year','purchase_month'],fill_value=0,margins=True,aggfunc=sum)

purchase_year  2018         2019                                               \
purchase_month   10  11  12    1   2   3   4   5    6    7   8    9   10   11   
Family                                                                          
X FIBER           0   4   8    7  10   2   8  13   17   17   4   20   17    2   
X MUSIC           3   2   3    4   7   2   0   6    0    0   0    5    0    0   
X PRIVACY         0   5   4    0  10  12   3  12   13    6   2    9    6    0   
X PROTECTION      0   0   4    2   5  25  13   4   29   20   5   19   28   28   
X SECURITY        0   0   2    2   0   0   0   5   13    5   0   53   28    2   
X UCOM            0  13  11   20  22  11  16  22   65   57  12   59   94   78   
All               3  24  32   35  54  52  40  62  137  105  23  165  173  110   

purchase_year       2020   All  
purchase_month   12    1        
Family                          
X FIBER           3    7   139  
X MUSIC           0    1    33  
X PRIVACY         2   11    95  
X PROTECTION      4    1   187  
X SECURITY        2    1   113  
X UCOM          120   51   651  
All             131   72  1218

In [16]:
ass_vta_canal_prod_mes = df[['isSold']].groupby(by=[df.purchase_date__c.dt.year,df.purchase_date__c.dt.month,df.canal_venta,df.Family,df.Velocity,df.isNeba]).sum()
ass_vta_canal_prod_mes.index.names = ['purchase_year', 'purchase_month','canal_venta','Family','Velocity','isNeba']
ass_vta_canal_prod_mes.reset_index(inplace=True)
ass_vta_canal_prod_mes.pivot_table('isSold',['canal_venta','Family','Velocity','isNeba'],['purchase_year','purchase_month'],fill_value=0,margins=True,aggfunc=sum).reset_index()

purchase_year     canal_venta        Family Velocity isNeba 2018         2019  \
purchase_month                                                10  11  12    1   
0                         DHO       X FIBER      100      0    0   0   1    1   
1                         DHO       X FIBER      100      1    0   0   0    1   
2                         DHO       X FIBER      200      0    0   0   0    0   
3                         DHO       X FIBER      200      1    0   0   0    0   
4                         DHO       X FIBER       50      0    0   2   1    2   
5                         DHO       X FIBER       50      1    0   1   1    0   
6                         DHO       X MUSIC               0    2   1   3    3   
7                         DHO     X PRIVACY      100      0    0   0   0    0   
8                         DHO     X PRIVACY      200      0    0   0   0    0   
9                         DHO     X PRIVACY       50      0    0   5   0    0   
10                        DHO  X PROTECTION               0    0   0   1    2   
11                        DHO    X SECURITY      100      0    0   0   0    0   
12                        DHO    X SECURITY      200      0    0   0   0    0   
13                        DHO        X UCOM               0    0   5   0   19   
14              IT integrator       X FIBER      100      0    0   0   3    2   
15              IT integrator       X FIBER      100      1    0   0   0    0   
16              IT integrator       X FIBER      200      0    0   0   1    0   
17              IT integrator       X FIBER      200      1    0   1   0    0   
18              IT integrator       X FIBER       50      0    0   0   0    0   
19              IT integrator       X FIBER       50      1    0   0   0    1   
20              IT integrator       X MUSIC               0    0   0   0    1   
21              IT integrator     X PRIVACY      200      0    0   0   1    0   
22              IT integrator     X PRIVACY       50      0    0   0   1    0   
23              IT integrator  X PROTECTION               0    0   0   0    0   
24              IT integrator    X SECURITY      100      0    0   0   2    2   
25              IT integrator    X SECURITY      200      0    0   0   0    0   
26              IT integrator        X UCOM               0    0   0   6    0   
27                        WEB       X FIBER      100      0    0   0   0    0   
28                        WEB       X FIBER      200      0    0   0   0    0   
29                        WEB       X FIBER      200      1    0   0   1    0   
30                        WEB       X FIBER       50      0    0   0   0    0   
31                        WEB       X MUSIC               0    1   1   0    0   
32                        WEB     X PRIVACY      100      0    0   0   0    0   
33                        WEB     X PRIVACY       50      0    0   0   2    0   
34                        WEB  X PROTECTION               0    0   0   3    0   
35                        WEB    X SECURITY      200      0    0   0   0    0   
36                        WEB        X UCOM               0    0   8   5    1   
37                        All                                  3  24  32   35   

purchase_year           ...                                       2020   All  
purchase_month   2   3  ...   5    6    7   8    9   10   11   12    1        
0                2   0  ...   2    0    1   0    2    0    0    0    0    10  
1                2   0  ...   1    0    1   1    0    0    0    0    0     6  
2                0   0  ...   6    0    1   0    2    0    0    1    0    10  
3                1   0  ...   0    4    4   0    0    0    0    0    0    10  
4                0   1  ...   0    1    3   1    0    0    0    0    0    11  
5                3   0  ...   0    0    3   1    0    0    1    0    0    10  
6                7   1  ...   0    0    0   0    3    0    0    0    1    21  
7                2   0  ...   0    3    0   0   

In [20]:
cli_vta_canal_mes = df[['cif']].groupby(by=[df.purchase_date__c.dt.year,df.purchase_date__c.dt.month,df.canal_venta]).nunique()
cli_vta_canal_mes.index.names = ['purchase_year', 'purchase_month', 'canal_venta']
cli_vta_canal_mes.reset_index(inplace=True)
cli_vta_canal_mes.pivot_table('cif',['canal_venta'],['purchase_year','purchase_month'],fill_value=0,margins=True,aggfunc=sum)

purchase_year  2018         2019                                              \
purchase_month   10  11  12    1   2   3   4   5   6   7   8   9  10  11  12   
canal_venta                                                                    
DHO               2   6   7    9  22  13   7  15  34  43  12  35  40  36  34   
IT integrator     0   1   4    3   1   1   8   3   5   4   1   6   2   2   1   
WEB               1   4   7    1   1   7   4   7   6   5   3   9   7   4   3   
All               3  11  18   13  24  21  19  25  45  52  16  50  49  42  38   

purchase_year  2020  All  
purchase_month    1       
canal_venta               
DHO              21  336  
IT integrator     1   43  
WEB               4   73  
All              26  452

In [18]:
cli_vta_canal_fam_mes = df[['cif']].groupby(by=[df.purchase_date__c.dt.year,df.purchase_date__c.dt.month,df.canal_venta,df.Family]).nunique()
cli_vta_canal_fam_mes.index.names = ['purchase_year', 'purchase_month', 'canal_venta','Family']
cli_vta_canal_fam_mes.reset_index(inplace=True)
cli_vta_canal_fam_mes.pivot_table('cif',['canal_venta','Family'],['purchase_year','purchase_month'],fill_value=0)

purchase_year              2018       2019                                     \
purchase_month               10 11 12   1  2  3  4  5   6   7  8   9   10  11   
canal_venta   Family                                                            
DHO           X FIBER         0  2  3    4  6  1  2  4   5  12  3   3   0   1   
              X MUSIC         2  1  3    1  4  1  0  0   0   0  0   1   0   0   
              X PRIVACY       0  2  0    0  4  6  1  2   5   3  1   3   2   0   
              X PROTECTION    0  0  1    2  5  4  3  4  14  17  5   9  16  10   
              X SECURITY      0  0  0    0  0  0  0  1   2   2  0   3   1   1   
              X UCOM          0  1  0    4  5  1  2  8  15  14  3  16  21  25   
IT integrator X FIBER         0  1  4    3  0  1  6  2   3   3  1   3   1   1   
              X MUSIC         0  0  0    1  0  0  0  1   0   0  0   1   0   0   
              X PRIVACY       0  0  1    0  0  0  0  0   0   0  0   0   0   0   
              X PROTECTION    0  0  0    0  0  0  3  0   0   0  0   2   0   1   
              X SECURITY      0  0  2    2  0  0  0  1   3   1  0   1   2   0   
              X UCOM          0  0  1    0  1  1  2  1   0   0  0   0   0   0   
WEB           X FIBER         0  0  1    0  1  0  0  0   1   1  0   2   0   0   
              X MUSIC         1  1  0    0  0  1  0  5   0   0  0   1   0   0   
              X PRIVACY       0  0  1    0  0  0  1  0   0   0  0   0   0   0   
              X PROTECTION    0  0  3    0  0  1  2  0   1   0  0   0   1   1   
              X SECURITY      0  0  0    0  0  0  0  0   0   0  0   0   0   0   
              X UCOM          0  3  2    1  0  5  1  2   4   4  3   6   6   3   

purchase_year                  2020  
purchase_month              12   1   
canal_venta   Family                 
DHO           X FIBER        1    0  
              X MUSIC        0    1  
              X PRIVACY      1    3  
              X PROTECTION   4    1  
              X SECURITY     1    0  
              X UCOM        27   16  
IT integrator X FIBER        1    1  
              X MUSIC        0    0  
              X PRIVACY      0    0  
              X PROTECTION   0    0  
              X SECURITY     0    0  
              X UCOM         0    0  
WEB           X FIBER        1    0  
              X MUSIC        0    0  
              X PRIVACY      0    0  
              X PROTECTION   0    0  
              X SECURITY     0    1  
              X UCOM         2    3

# Altas

In [18]:
df2 = Cli_altas_bajas(df)

In [19]:
df2.head()

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isClient,isActive,assets_dea,isBaja
0,03817373V,WEB,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,1,0,2,1
1,07477239P,DHO,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,1,0,1,1
2,11058321J,DHO,1,2019-01-23 00:00:00,NaT,1,1,1,0,0
3,12371541W,WEB,0,2018-12-18 00:00:01,2019-01-02 12:15:09,1,1,0,1,1
4,16601899Q,DHO,1,2019-05-22 00:00:01,NaT,5,1,1,4,0


In [23]:
a = Cli_altas_bajas_mes_canal(df2)

In [24]:
a[((a['year']==2019)|(a['year']==2020))&((a['month']==12)|(a['month']==1))]

,year,month,canal,isClient,isBaja
8,2019,1,DHO,7.0,1.0
9,2019,1,IT integrator,3.0,0.0
10,2019,1,WEB,3.0,2.0
38,2019,12,DHO,24.0,6.0
39,2019,12,WEB,1.0,1.0
40,2020,1,DHO,22.0,7.0
41,2020,1,WEB,3.0,0.0
